# 1. Carga de Datos

Leemos desde market.db

In [1]:
library('DBI')
library('RSQLite')
library('magrittr')
library('dplyr')
library('lubridate')
library('tidyr')
library('ggplot2')


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




Attaching package: ‘lubridate’

The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Attaching package: ‘tidyr’



Attaching package: ‘lubridate’

The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Attaching package: ‘tidyr’



The following object is masked from ‘package:magrittr’:

    extract

The following object is masked from ‘package:magrittr’:

    extract



In [2]:
con = dbConnect(RSQLite::SQLite(), "../../01_Datos/market.db")

In [3]:
dbListTables(con)

[1] "calendar"    "sales"       "sell_prices"

Vamos a leer las tablas y ver el contenido.

In [4]:
calendar = dbReadTable(con, "calendar")
sales = dbReadTable(con, "sales")
sell_prices = dbReadTable(con, "sell_prices")

In [5]:
calendar %>% head()

  index       date wm_yr_wk   weekday wday month year     d event_name_1 event_type_1 event_name_2 event_type_2
1     0 2013-01-01    11249   Tuesday    4     1 2013 d_704      NewYear     National         <NA>         <NA>
2     1 2013-01-02    11249 Wednesday    5     1 2013 d_705         <NA>         <NA>         <NA>         <NA>
3     2 2013-01-03    11249  Thursday    6     1 2013 d_706         <NA>         <NA>         <NA>         <NA>
4     3 2013-01-04    11249    Friday    7     1 2013 d_707         <NA>         <NA>         <NA>         <NA>
5     4 2013-01-05    11250  Saturday    1     1 2013 d_708         <NA>         <NA>         <NA>         <NA>
6     5 2013-01-06    11250    Sunday    2     1 2013 d_709         <NA>         <NA>         <NA>         <NA>

In [6]:
sales %>% head()

     index id item_id dept_id cat_id store_id state_id d_704 d_705 d_706 d_707 d_708 d_709 d_710 d_711 d_712 d_713 d_714 d_715 d_716
     d_717 d_718 d_719 d_720 d_721 d_722 d_723 d_724 d_725 d_726 d_727 d_728 d_729 d_730 d_731 d_732 d_733 d_734 d_735 d_736 d_737 d_738
     d_739 d_740 d_741 d_742 d_743 d_744 d_745 d_746 d_747 d_748 d_749 d_750 d_751 d_752 d_753 d_754 d_755 d_756 d_757 d_758 d_759 d_760
     d_761 d_762 d_763 d_764 d_765 d_766 d_767 d_768 d_769 d_770 d_771 d_772 d_773 d_774 d_775 d_776 d_777 d_778 d_779 d_780 d_781 d_782
     d_783 d_784 d_785 d_786 d_787 d_788 d_789 d_790 d_791 d_792 d_793 d_794 d_795 d_796 d_797 d_798 d_799 d_800 d_801 d_802 d_803 d_804
     d_805 d_806 d_807 d_808 d_809 d_810 d_811 d_812 d_813 d_814 d_815 d_816 d_817 d_818 d_819 d_820 d_821 d_822 d_823 d_824 d_825 d_826
     d_827 d_828 d_829 d_830 d_831 d_832 d_833 d_834 d_835 d_836 d_837 d_838 d_839 d_840 d_841 d_842 d_843 d_844 d_845 d_846 d_847 d_848
     d_849 d_850 d_851 d_852 d_853 d_854 d_85

In [7]:
sell_prices %>% head()

    index store_id     item_id wm_yr_wk sell_price
1 1862524     CA_3 FOODS_3_090    11249       1.25
2 1862525     CA_3 FOODS_3_090    11250       1.25
3 1862526     CA_3 FOODS_3_090    11251       1.25
4 1862527     CA_3 FOODS_3_090    11252       1.25
5 1862528     CA_3 FOODS_3_090    11301       1.38
6 1862529     CA_3 FOODS_3_090    11302       1.38

En ambas tres tablas, eliminamos la columna index porque no es significativa. De sales también eliminamos id porque es información redundante.

In [8]:
calendar = calendar %>% dplyr::select(-index)
sales = sales %>% dplyr::select(-index, -id)
sell_prices = sell_prices %>% dplyr::select(-index)

Ahora, haremos pasaremos de formato transaccional a tabular para normalizar algunas tablas. En sales, la estructura viene como los días de ventas y las ventas por columnas, las pasamos a filas.

In [9]:
sales = pivot_longer(sales, 
                      cols = starts_with("d_"), 
                      names_to = "d", 
                      values_to = "ventas")

Unimos las tablas por los campos comunes, en primer lugar sales y calendar por el campo en común "d" que hemos transformado en sales en el anterior paso.

In [10]:
df = sales %>% left_join(calendar, by = "d")

Visualizamos:

In [11]:
df %>%  head()

# A tibble: 6 × 17
  item_id   dept_id cat_id store_id state_id d     ventas date  wm_yr_wk weekday  wday month  year event_name_1 event_type_1 event_name_2
  <chr>     <chr>   <chr>  <chr>    <chr>    <chr>  <int> <chr>    <int> <chr>   <int> <int> <int> <chr>        <chr>        <chr>       
1 FOODS_3_… FOODS_3 FOODS  CA_3     CA       d_704      0 2013…    11249 Tuesday     4     1  2013 NewYear      National     NA          
2 FOODS_3_… FOODS_3 FOODS  CA_3     CA       d_705    224 2013…    11249 Wednes…     5     1  2013 NA           NA           NA          
3 FOODS_3_… FOODS_3 FOODS  CA_3     CA       d_706    241 2013…    11249 Thursd…     6     1  2013 NA           NA           NA          
4 FOODS_3_… FOODS_3 FOODS  CA_3     CA       d_707    232 2013…    11249 Friday      7     1  2013 NA           NA           NA          
5 FOODS_3_… FOODS_3 FOODS  CA_3     CA       d_708    301 2013…    11250 Saturd…     1     1  2013 NA           NA           NA          
6 FOODS_3_… FOO

Actualizamos con sell_prices, que tiene en común con df los campos "store_id", "item_id" y "wm-yr_wk"

In [12]:
df = df %>% left_join(sell_prices, by = c("store_id", "item_id", "wm_yr_wk"))

Vamos a comprobar que todos los campos se han integrado correctamente.

In [13]:
df %>% select(store_id, item_id, wm_yr_wk, d, sell_price) %>%
arrange(store_id, item_id, wm_yr_wk, d) %>% 
head(10)

# A tibble: 10 × 5
   store_id item_id     wm_yr_wk d     sell_price
   <chr>    <chr>          <int> <chr>      <dbl>
 1 CA_3     FOODS_3_090    11249 d_704       1.25
 2 CA_3     FOODS_3_090    11249 d_705       1.25
 3 CA_3     FOODS_3_090    11249 d_706       1.25
 4 CA_3     FOODS_3_090    11249 d_707       1.25
 5 CA_3     FOODS_3_090    11250 d_708       1.25
 6 CA_3     FOODS_3_090    11250 d_709       1.25
 7 CA_3     FOODS_3_090    11250 d_710       1.25
 8 CA_3     FOODS_3_090    11250 d_711       1.25
 9 CA_3     FOODS_3_090    11250 d_712       1.25
10 CA_3     FOODS_3_090    11250 d_713       1.25

Vamos ahora a poner las columnas en un orden más adecuado.

In [14]:
columns_order = c('date', 'state_id', 'store_id', 'dept_id', 'cat_id', 'item_id', 'wm_yr_wk', 'd', 'ventas',
                      'sell_price', 'year', 'month', 'wday', 'weekday', 'event_name_1', 'event_type_1', 'event_name_2',
                      'event_type_2')

In [15]:
df = df[, columns_order]

In [16]:
df %>% head()

# A tibble: 6 × 18
  date      state_id store_id dept_id cat_id item_id wm_yr_wk d     ventas sell_price  year month  wday weekday event_name_1 event_type_1
  <chr>     <chr>    <chr>    <chr>   <chr>  <chr>      <int> <chr>  <int>      <dbl> <int> <int> <int> <chr>   <chr>        <chr>       
1 2013-01-… CA       CA_3     FOODS_3 FOODS  FOODS_…    11249 d_704      0       1.25  2013     1     4 Tuesday NewYear      National    
2 2013-01-… CA       CA_3     FOODS_3 FOODS  FOODS_…    11249 d_705    224       1.25  2013     1     5 Wednes… NA           NA          
3 2013-01-… CA       CA_3     FOODS_3 FOODS  FOODS_…    11249 d_706    241       1.25  2013     1     6 Thursd… NA           NA          
4 2013-01-… CA       CA_3     FOODS_3 FOODS  FOODS_…    11249 d_707    232       1.25  2013     1     7 Friday  NA           NA          
5 2013-01-… CA       CA_3     FOODS_3 FOODS  FOODS_…    11250 d_708    301       1.25  2013     1     1 Saturd… NA           NA          
6 2013-01-… CA 

Vamos a establecer la columna date, como datetime.

In [17]:
df$date = ymd(df$date)

Guardamos el dataset en formato .rds.

In [18]:
saveRDS(df, file = "../02_Variables/df.rds")